In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from datafed.CommandLib import API
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import random

In [ ]:
# Creates Folder for Dataset
try:
    # Attempt to create a directory named 'datapath'
    datapath = os.mkdir("./datapath")
    true_path = os.path.abspath(datapath)  # Get the absolute path of the created directory
except:
    # If the directory already exists, just use its path
    datapath = "./datapath"
    true_path = os.path.abspath(datapath)  # Get the absolute path of the existing directory



In [ ]:
# Initialize the API object
df_api = API()

# Use the dataGet method to download data
# "d/525645423" is the dataset identifier
# os.path.abspath(datapath) provides the absolute path to the datapath directory
# wait=True ensures the function waits for the download to complete
dget_resp = df_api.dataGet("d/525645998", os.path.abspath(datapath), wait=True)
dget_resp

In [ ]:
def unzip_folder(folder_path, zip_file_name, extract_to=None):
    """
    Unzips a .zip file in the given folder and deletes the .zip file afterwards.
    
    folder_path: Path to the folder containing the .zip file.
    zip_file_name: Name of the .zip file (with extension).
    extract_to: Path to extract the contents to (default: same as folder_path).
    """
    zip_path = os.path.join(folder_path, zip_file_name)  # Full path to the zip file
    
    # Default extraction path is the folder containing the zip file
    if extract_to is None:
        extract_to = folder_path
    
    # Check if the .zip file exists
    if not os.path.exists(zip_path):
        raise FileNotFoundError(f"The file {zip_file_name} was not found in {folder_path}")
    
    # Unzipping the file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"Extracted {zip_file_name} to {extract_to}")
    
    try:
        os.remove(zip_path)
        print(f"Deleted the zip file: {zip_file_name}")
    except OSError as e:
        print(f"Error while deleting the zip file: {e}")

# Unzip the dataset
zip_file_name = "525645998.zip"     # Name of Dataset Zipfile
unzip_folder(true_path, zip_file_name) #Unzip Dataset
images_path = os.path.join(true_path, "Pokemon_Dataset_Correct") #Creates absolute path to Dataset

In [ ]:
# Define constants
IMAGE_DIR = images_path #
IMG_SIZE = (255, 255) 
BATCH_SIZE = 32 #Number of images to process at a time

In [ ]:
# Function to parse labels from filenames
def parse_filename(filename):
    """
    Parses the filename to extract metadata.

    Args:
        filename (str): The name of the image file.

    Returns:
        pokemon_name (str): Name of Pokemon
        shiny_form (int): 1 if Pokemon is shiny, 0 if Pokemon is Normal
        gender (str): Gender of Pokemon

    """
    parts = filename.replace('.jpg', '').split(' ')  # Remove the file extension and split by spaces
    location_name = parts[0]  # Extract the location and name part
    shiny = 1 if 'Shiny' in parts else 0  # Determine if the Pokémon is shiny
    gender = 'Unknown'  # Default gender
    if 'Male & Female' in filename:
        gender = 'Male & Female'  # Check for both genders
    elif 'Male' in filename:
        gender = 'Male'  # Check for male gender
    elif 'Female' in filename:
        gender = 'Female'  # Check for female gender
    
    location, name = location_name.split('_', 1)  # Split location and name
    
    return name, shiny, gender  # Return the parsed values

In [ ]:
# Load dataset
data = []  # List to store file paths of images
labels_name = []  # List to store Pokémon names
labels_shiny = []  # List to store shiny status (1 for shiny, 0 for normal)
labels_gender = []  # List to store gender of Pokémon

# Iterate over each file in the image directory
for file in os.listdir(IMAGE_DIR):
    if file.endswith(".jpg"):  # Check if the file is a JPEG image
        filepath = os.path.join(IMAGE_DIR, file)  # Get the full path of the image file
        name, shiny, gender = parse_filename(file)  # Parse the filename to extract metadata
        data.append(filepath)  # Add the file path to the data list
        labels_name.append(name)  # Add the Pokémon name to the labels_name list
        labels_shiny.append(shiny)  # Add the shiny status to the labels_shiny list
        labels_gender.append(gender)  # Add the gender to the labels_gender list

In [ ]:
# Preprocess labels
unique_names = sorted(set(labels_name))  # Get unique Pokémon names and sort them
unique_genders = ["Male", "Female", "Male & Female"]  # Define unique gender labels

# Create dictionaries to map names and genders to indices
name_to_idx = {name: i for i, name in enumerate(unique_names)}
gender_to_idx = {gender: i for i, gender in enumerate(unique_genders)}

# Convert labels to indices
y_name = [name_to_idx[name] for name in labels_name]  # Convert Pokémon names to indices
y_shiny = labels_shiny  # Shiny status remains the same
y_gender = [gender_to_idx[gender] for gender in labels_gender]  # Convert genders to indices

# Split data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(
    data, list(zip(y_name, y_shiny, y_gender)), test_size=0.2, random_state=42
)

In [ ]:
# Preprocess images
#Initialize Augmentatio Parameters
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)



def preprocess_images(filepaths, labels, batch_size):
    """
    Preprocesses images and labels for a machine learning model.

    This function creates a TensorFlow dataset from a list of image file paths and corresponding labels.
    It loads each image, resizes it with padding, normalizes pixel values, and structures the labels
    into a dictionary format suitable for model outputs.

    Args:
        filepaths (list of str): List of file paths to the images.
        labels (list of tuples): List of tuples where each tuple contains three elements:
            - name_output (int): The label for the name output.
            - shiny_output (int): The label for the shiny output.
            - gender_output (int): The label for the gender output.
        batch_size (int): The size of the batches to be generated.

    Returns:
        tf.data.Dataset: A TensorFlow dataset yielding batches of images and corresponding label dictionaries.
    """
    def generator():
        for filepath, label in zip(filepaths, labels):
            # Load the image
            image = tf.keras.utils.load_img(filepath)
            image = tf.keras.utils.img_to_array(image) / 255.0 #Normalize pixel values
            # Resize with padding
            image = tf.image.resize_with_pad(image, target_height=IMG_SIZE[0], target_width=IMG_SIZE[1])
            # Restructure labels into a dictionary for model outputs
            label_dict = {
                "name_output": label[0],
                "shiny_output": label[1],
                "gender_output": label[2]
            }
            yield image, label_dict  # Yield the image and label dictionary
    return tf.data.Dataset.from_generator(
        generator,  # Use the generator function to create the dataset
        output_signature=(
            tf.TensorSpec(shape=(*IMG_SIZE, 3), dtype=tf.float32),  # Define the shape and type of the image tensor
            {
                "name_output": tf.TensorSpec(shape=(), dtype=tf.int32),  # Define the shape and type of the name output
                "shiny_output": tf.TensorSpec(shape=(), dtype=tf.int32),  # Define the shape and type of the shiny output
                "gender_output": tf.TensorSpec(shape=(), dtype=tf.int32),  # Define the shape and type of the gender output
            },
        )
    ).batch(batch_size)  # Batch the dataset with the specified batch size

train_dataset = preprocess_images(train_data, train_labels, BATCH_SIZE) #Create training dataset
val_dataset = preprocess_images(val_data, val_labels, BATCH_SIZE)     #Creat Validation dataset


NameError: name 'train_data' is not defined

In [ ]:
def visualize_preprocessed_images(dataset, num_images=5):
    """
    Visualizes a few random preprocessed images from the dataset.

    Args:
        dataset: A TensorFlow Dataset object containing preprocessed images and labels.
        num_images: Number of images to visualize.
    """
    plt.figure(figsize=(15, 5))
    for image_batch, label_batch in dataset.take(1):  # Take one batch from the dataset
        total_images = image_batch.shape[0]
        random_indices = random.sample(range(total_images), num_images)  # Randomly select indices
        for i, idx in enumerate(random_indices):
            plt.subplot(1, num_images, i + 1)
            plt.imshow(image_batch[idx].numpy())
            plt.axis('off')
            # Display the labels for the selected image
            label = label_batch
            name = label["name_output"][idx].numpy()
            shiny = "Shiny" if label["shiny_output"][idx].numpy() == 1 else "Normal"
            gender = unique_genders[label["gender_output"][idx].numpy()]
            plt.title(f"{unique_names[name]}\n{shiny}, {gender}")
        break  # Only process the first batch
    plt.tight_layout()
    plt.show()

visualize_preprocessed_images(train_dataset)

In [ ]:
# Build the model
# Load the EfficientNetB0 model without the top classification layer
base_model = EfficientNetB0(include_top=False, input_shape=(*IMG_SIZE, 3))

# Flatten the output layer to 1 dimension
x = Flatten()(base_model.output)

# Add a dropout layer to reduce overfitting
x = Dropout(0.5)(x)  # First dropout

# Add a fully connected layer with ReLU activation
x = Dense(512, activation='relu')(x)

# Add another dropout layer to reduce overfitting
x = Dropout(0.3)(x)


# Add regularization to the Dense layers
name_output = Dense(len(unique_names), activation="softmax", name="name_output",
                    kernel_regularizer=l2(0.02))(x)  # Output layer for Pokémon names with L2 regularization
shiny_output = Dense(1, activation="sigmoid", name="shiny_output",
                     kernel_regularizer=l2(0.02))(x)  # Output layer for shiny status with L2 regularization
gender_output = Dense(len(unique_genders), activation="softmax", name="gender_output",
                      kernel_regularizer=l2(0.02))(x)  # Output layer for gender with L2 regularization

# Final model
model = Model(inputs=base_model.input, outputs=[name_output, shiny_output, gender_output])  # Create the model

In [ ]:
# Train the model
# Compile the model with appropriate loss functions and metrics
model.compile(
    optimizer="adam",  # Use Adam optimizer
    loss={
        "name_output": "sparse_categorical_crossentropy",  # Loss for name classification
        "shiny_output": "binary_crossentropy",  # Loss for shiny status classification
        "gender_output": "sparse_categorical_crossentropy",  # Loss for gender classification
    },
    metrics={
        "name_output": ["accuracy"],  # Accuracy metric for name classification
        "shiny_output": ["accuracy"],  # Accuracy metric for shiny status classification
        "gender_output": ["accuracy"],  # Accuracy metric for gender classification
    }
)

# Early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",  # Monitor validation loss
    patience=3,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

# Reduce learning rate when a metric has stopped improving
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",  # Monitor validation loss
    factor=0.2,  # Factor by which the learning rate will be reduced
    patience=2,  # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6  # Lower bound on the learning rate
)


# Train the model and store the training history
history = model.fit(
    train_dataset,  # Training dataset
    validation_data=val_dataset,  # Validation dataset
    epochs=50,  # Number of epochs to train
    callbacks=[early_stopping, reduce_lr],  # List of callbacks for early stopping and learning rate reduction
    verbose=1  # Verbosity mode (1 = progress bar)
)


NameError: name 'model' is not defined

In [ ]:
#Save the entire model
model.save("pokemon_identifier_model.h5")

In [ ]:
def plot_training_history(history):
    """
    Plots the training and validation accuracy for different outputs and the learning rate over epochs.

    Parameters:
    history (keras.callbacks.History): A History object returned by the fit method of a Keras model. 
                                       It contains the training and validation metrics for each epoch.

    The function will plot:
    - Training and validation accuracy for 'name_output', 'shiny_output', and 'gender_output'.
    - Learning rate over epochs if 'learning_rate' is present in the history.

    Returns:
    None
    """
    # Extract history metrics
    history_dict = history.history

    # Outputs to plot
    outputs = ['name_output', 'shiny_output', 'gender_output']

    # Create a separate plot for each output
    for output in outputs:
        train_acc = history_dict[f"{output}_accuracy"]  # Training accuracy for the current output
        val_acc = history_dict[f"val_{output}_accuracy"]  # Validation accuracy for the current output

        plt.figure(figsize=(10, 6))  # Set the figure size
        plt.plot(train_acc, label=f"Train {output} Accuracy")  # Plot training accuracy
        plt.plot(val_acc, label=f"Validation {output} Accuracy")  # Plot validation accuracy
        plt.title(f"Training vs. Validation Accuracy for {output.capitalize()}")  # Set the title
        plt.xlabel("Epochs")  # Set the x-axis label
        plt.ylabel("Accuracy")  # Set the y-axis label
        plt.legend()  # Show the legend
        plt.grid(True)  # Show the grid
        plt.show()  # Display the plot

    # Plot learning rate vs. epochs if learning rate is present in the history
    if 'learning_rate' in history_dict:
        learning_rate = history_dict['learning_rate']  # Extract learning rate
        plt.figure(figsize=(10, 6))  # Set the figure size
        plt.plot(learning_rate, label="Learning Rate")  # Plot learning rate
        plt.title("Learning Rate vs. Epochs")  # Set the title
        plt.xlabel("Epochs")  # Set the x-axis label
        plt.ylabel("Learning Rate")  # Set the y-axis label
        plt.legend()  # Show the legend
        plt.grid(True)  # Show the grid
        plt.show()  # Display the plot

# Call the function with your history object
plot_training_history(history)



In [ ]:
# Function to predict on a new image
def predict_from_image(filepath, model, img_size, class_names, gender_labels):
    """
    Predicts the Pokémon name, shiny status, and gender from an image, and visualizes preprocessing.

    Args:
        filepath (str): Path to the image file.
        model (tf.keras.Model): Trained model for predictions.
        img_size (tuple): Image size expected by the model (height, width).
        class_names (list): List of Pokémon names corresponding to indices.
        gender_labels (list): List of gender labels corresponding to indices.

    Returns:
        dict: Prediction results containing name, shiny status, and gender.
    """
    # Load the image
    image = tf.keras.utils.load_img(filepath)
    image_array = tf.keras.utils.img_to_array(image) / 255.0  # Normalize pixel values
    
    # Resize with padding (to preserve aspect ratio)
    preprocessed_image = tf.image.resize_with_pad(image_array, img_size[0], img_size[1])
    
    # Add batch dimension
    image_batch = tf.expand_dims(preprocessed_image, axis=0)

    # Predict
    predictions = model.predict(image_batch)

    # Decode predictions
    name_idx = tf.argmax(predictions[0], axis=1).numpy()[0]  # Get the index of the predicted Pokémon name
    shiny_status = int(predictions[1][0] > 0.5)  # Determine shiny status (binary classification)
    gender_idx = tf.argmax(predictions[2], axis=1).numpy()[0]  # Get the index of the predicted gender

    # Prepare prediction results
    result = {
        "name": class_names[name_idx],
        "shiny": "Shiny" if shiny_status == 1 else "Normal",
        "gender": gender_labels[gender_idx],
    }

    # Display prediction on the image
    plt.figure(figsize=(6, 6))
    plt.imshow(preprocessed_image.numpy())
    plt.axis('off')
    plt.title(
        f"Name: {result['name']}\n"
        f"Shiny Status: {result['shiny']}\n"
        f"Gender: {result['gender']}"
    )
    plt.show()

    return result

# Testing Real Image
dget_resp = df_api.dataGet("d/525646265", os.path.abspath(datapath), wait=True)  # Download the image
dget_resp
actualImage_path = os.path.join(true_path, "525646265.jpg")  # Get the path to the downloaded image
print(actualImage_path)

# Predict and display the results
prediction = predict_from_image(actualImage_path, model, IMG_SIZE, unique_names, unique_genders)
